In [ ]:
import cabinetry

We customize the output from `cabinetry` via a helper function. This is optional, and the `logging` module can be used directly as well to further customize the behavior.

In [ ]:
cabinetry.set_logging()

Download a workspace from HEPData, extract it, pick a signal with `pyhf`. We use a workspace from an ATLAS search for bottom-squark pair production: [JHEP 12 (2019) 060](https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/PAPERS/SUSY-2018-31/). The corresponding HEPData entry is [10.17182/hepdata.89408.v3](https://doi.org/10.17182/hepdata.89408.v3).

In [ ]:
import json
import pyhf
from pyhf.contrib.utils import download

download("https://www.hepdata.net/record/resource/1935437?view=true", "bottom-squarks")
ws = pyhf.Workspace(json.load(open("bottom-squarks/RegionC/BkgOnly.json")))
patchset = pyhf.PatchSet(json.load(open("bottom-squarks/RegionC/patchset.json")))
ws = patchset.apply(ws, "sbottom_600_280_150")
cabinetry.workspace.save(ws, "bottom-squarks.json")

The `bottom-squarks.json` workspace is now ready to be used. We will run a maximum likelihood fit with `cabinetry` and visualize the results. First, we have a brief look at the content of the workspace:

In [ ]:
!pyhf inspect bottom-squarks.json | head -n 6

The fit model specified in the workspace is created next.

In [ ]:
ws = cabinetry.workspace.load("bottom-squarks.json")
model, data = cabinetry.model_utils.model_and_data(ws)

We can take a look at a yield table for this model. We first generate the pre-fit model prediction, and then pass it to a function to produce a yield table from it.

In [ ]:
model_prefit = cabinetry.model_utils.prediction(model)
cabinetry.tabulate.yields(model_prefit, data)

We can also visualize the pre-fit model prediction and compare it to data. the `visualize.data_mc` function returns a list of dictionaries containing the `matplotlib` figures, which we could use to customize them as needed. We do not need to customize anything here, so we assign the return value to `_`.

In [ ]:
_ = cabinetry.visualize.data_mc(model_prefit, data)

Next up is a maximum likelihood fit:

In [ ]:
fit_results = cabinetry.fit.fit(model, data)

We can now visualize the post-fit distributions. To do so, we need a post-fit model prediction. It is obtained like the pre-fit model prediction, but this time with an additional argument to pass in the fit results.

In [ ]:
model_postfit = cabinetry.model_utils.prediction(model, fit_results=fit_results)
_ = cabinetry.visualize.data_mc(model_postfit, data)

The nuisance parameter pulls and correlations are visualized below.

In [ ]:
cabinetry.visualize.pulls(fit_results, exclude="mu_SIG")

In [ ]:
cabinetry.visualize.correlation_matrix(fit_results, pruning_threshold=0.2)